In [46]:
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
import chart_studio.plotly as py
import chart_studio
import matplotlib.pyplot as plt

chart_studio.tools.set_credentials_file(username='oystein.nerva', api_key='0ODcZ6UisJ8n0AjEQt9N')

In [101]:
# finn API url her: https://data.ssb.no/api/v0/dataset

url = "https://data.ssb.no/api/v0/dataset/1080.csv?lang=no"

df = pd.read_csv(url, sep=";", decimal=",", encoding="ISO-8859-1")
df.head()

,region,alder,år,statistikkvariabel,"07459: Befolkning, etter region, alder, år og statistikkvariabel"
0,3001 Halden,000 0 år,2020,Personer,256
1,3001 Halden,001 1 år,2020,Personer,256
2,3001 Halden,002 2 år,2020,Personer,322
3,3001 Halden,003 3 år,2020,Personer,321
4,3001 Halden,004 4 år,2020,Personer,320


In [102]:
df["age"], df["rest"] = df["alder"].str.split(" ",1).str
df = df.drop(columns = ["rest", "statistikkvariabel", "år", "alder"])
df["age"] = pd.to_numeric(df["age"], errors='coerce')
df.head()

,region,"07459: Befolkning, etter region, alder, år og statistikkvariabel",age
0,3001 Halden,256,0.0
1,3001 Halden,256,1.0
2,3001 Halden,322,2.0
3,3001 Halden,321,3.0
4,3001 Halden,320,4.0


In [103]:
df["gammel"] = df.apply(lambda x: x["07459: Befolkning, etter region, alder, år og statistikkvariabel"] if x["age"] >= 65 else 0, axis=1)
df = df.groupby(["region"]).sum().reset_index()
df.head()

,region,"07459: Befolkning, etter region, alder, år og statistikkvariabel",age,gammel
0,0301 Oslo kommune,693494,5460.0,87347
1,1101 Eigersund,14811,5460.0,2687
2,1103 Stavanger,143574,5460.0,20728
3,1106 Haugesund,37357,5460.0,6342
4,1108 Sandnes,79537,5460.0,10291


In [104]:
df["Andel over 65 år"] = (df["gammel"]/df["07459: Befolkning, etter region, alder, år og statistikkvariabel"])*100
df.head()

,region,"07459: Befolkning, etter region, alder, år og statistikkvariabel",age,gammel,Andel over 65 år
0,0301 Oslo kommune,693494,5460.0,87347,12.595206
1,1101 Eigersund,14811,5460.0,2687,18.141922
2,1103 Stavanger,143574,5460.0,20728,14.437154
3,1106 Haugesund,37357,5460.0,6342,16.976738
4,1108 Sandnes,79537,5460.0,10291,12.938632


In [105]:
df["Kommunenr"], df["Kommune"] = df["region"].str.split(" ",1).str
df = df.drop(columns="region")
df.head()

,"07459: Befolkning, etter region, alder, år og statistikkvariabel",age,gammel,Andel over 65 år,Kommunenr,Kommune
0,693494,5460.0,87347,12.595206,0301,Oslo kommune
1,14811,5460.0,2687,18.141922,1101,Eigersund
2,143574,5460.0,20728,14.437154,1103,Stavanger
3,37357,5460.0,6342,16.976738,1106,Haugesund
4,79537,5460.0,10291,12.938632,1108,Sandnes


# kategoriser kommuner

In [106]:
kommuner = pd.read_excel("Kommunetilhorighet.xlsx")
kommuner.head()

,Kommunenr,Kommune,Foretak,Foretak kort
0,1835,TRÆNA,Helgelandssykehuset HF,HSYK
1,5425,Storfjord,UNN HF,UNN
2,1812,SØMNA,Helgelandssykehuset HF,HSYK
3,5406,Hammerfest,Finnmarkssykehuset HF,FIN
4,5418,Målselv,UNN HF,UNN


In [107]:
df["Kommunenr"] = pd.to_numeric(df["Kommunenr"], errors='coerce')
df = df.merge(kommuner, how="left", on="Kommunenr")
df.head()

,"07459: Befolkning, etter region, alder, år og statistikkvariabel",age,gammel,Andel over 65 år,Kommunenr,Kommune_x,Kommune_y,Foretak,Foretak kort
0,693494,5460.0,87347,12.595206,301,Oslo kommune,NaN,NaN,NaN
1,14811,5460.0,2687,18.141922,1101,Eigersund,NaN,NaN,NaN
2,143574,5460.0,20728,14.437154,1103,Stavanger,NaN,NaN,NaN
3,37357,5460.0,6342,16.976738,1106,Haugesund,NaN,NaN,NaN
4,79537,5460.0,10291,12.938632,1108,Sandnes,NaN,NaN,NaN


In [111]:
bars = df.sort_values(by=['Andel over 65 år'], ascending=True)
bars = bars.replace(np.nan, "Annet")
bars = bars.tail(20)

In [113]:
legend = [i for i in df["Kommune_x"].dropna().unique()]
legend.sort()

fig = px.bar(bars, 
             x='Andel over 65 år', y='Kommune_x', 
             orientation='h',
             category_orders={"Foretak kort": legend},
             height=500,
             title = "<b>Andel av befolkningen over 65 år</b><br>Datakilde: ssb.no.",
             color='Foretak',
             hover_name="Kommune_x",
             hover_data=["Andel over 65 år"],
             template = "plotly_white", 
             #labels = {"Vekstrate": "daglig vekst (%)", "- 1 uke": "Daglig vekst for en uke siden", "text": "Dager før dobling med dagens vekst"},
            )


fig.layout.coloraxis.showscale = False

fig.update_layout(
    yaxis=dict(
        separatethousands=True,
        categoryorder = 'total ascending',
        title_text="",
        fixedrange=True,
    ), xaxis=dict(fixedrange=True,
                 title_text=""),
    title_font=dict(family="Helvetica", size=18),
        margin=dict(b=150)) #bottom margin

fig.update_layout(autosize=True,
    margin=dict(l=100, r=100, t=150, b=0),
    paper_bgcolor='rgba(0, 0, 0, 0)'
    #plot_bgcolor='rgb(248, 248, 255)'    
                 )

#plotly.offline.plot(fig, filename='COVID_moment_fylke.html')
#py.plot(fig, filename='COVID19_vekstmomentum_fylker.html', auto_open=False)
fig.show()

# legg til smittetall fra VG

In [114]:
smittede = pd.read_excel("VG-smitte.xlsx")
smittede = smittede[["KOMMUNE_ID", "KOMMUNENAVN", "VG-tall 31.03 2030"]]
smittede.head()

,KOMMUNE_ID,KOMMUNENAVN,VG-tall 31.03 2030
0,1820,Alstahaug,3
1,5403,Alta,5
2,3428,Alvdal,1
3,4631,Alver,15
4,1871,Andøy,0


In [115]:
df = df.merge(smittede, how="left", left_on="Kommunenr", right_on="KOMMUNE_ID")
df.head()

,"07459: Befolkning, etter region, alder, år og statistikkvariabel",age,gammel,Andel over 65 år,Kommunenr,Kommune_x,Kommune_y,Foretak,Foretak kort,KOMMUNE_ID,KOMMUNENAVN,VG-tall 31.03 2030
0,693494,5460.0,87347,12.595206,301,Oslo kommune,NaN,NaN,NaN,301,Oslo,1233
1,14811,5460.0,2687,18.141922,1101,Eigersund,NaN,NaN,NaN,1101,Eigersund,14
2,143574,5460.0,20728,14.437154,1103,Stavanger,NaN,NaN,NaN,1103,Stavanger,120
3,37357,5460.0,6342,16.976738,1106,Haugesund,NaN,NaN,NaN,1106,Haugesund,34
4,79537,5460.0,10291,12.938632,1108,Sandnes,NaN,NaN,NaN,1108,Sandnes,41


In [116]:
fig = px.scatter(df.replace(np.nan, "Annet"), x="Andel over 65 år", y="VG-tall 31.03 2030", 
                 size="07459: Befolkning, etter region, alder, år og statistikkvariabel", color="Foretak",
                 #hover_name="country", 
                 log_y=True, log_x=True, 
                 size_max=50
                )
fig.show()

# Fylkesdata

In [117]:
url = "https://data.ssb.no/api/v0/dataset/1076.csv?lang=no"
df = pd.read_csv(url, sep=";", decimal=",", encoding="ISO-8859-1")

df["age"], df["rest"] = df["alder"].str.split(" ",1).str
df = df.drop(columns = ["rest", "statistikkvariabel", "år", "alder"])
df["age"] = pd.to_numeric(df["age"], errors='coerce')

df["gammel"] = df.apply(lambda x: x["07459: Befolkning, etter region, alder, år og statistikkvariabel"] if x["age"] >= 65 else 0, axis=1)
df = df.groupby(["region"]).sum().reset_index()

df["Andel over 65 år"] = (df["gammel"]/df["07459: Befolkning, etter region, alder, år og statistikkvariabel"])*100

df.head()

,region,"07459: Befolkning, etter region, alder, år og statistikkvariabel",age,gammel,Andel over 65 år
0,03 Oslo,693494,5460.0,87347,12.595206
1,11 Rogaland,479892,5460.0,71967,14.996499
2,15 Møre og Romsdal,265238,5460.0,52367,19.743400
3,18 Nordland,241235,5460.0,50191,20.805853
4,21 Svalbard,0,5460.0,0,NaN


In [118]:
df["Fylkesnr"], df["Fylke"] = df["region"].str.split(" ",1).str
df = df.drop(columns="region")
df["Fylkesnr"] = pd.to_numeric(df["Fylkesnr"], errors='coerce')
df.head()

,"07459: Befolkning, etter region, alder, år og statistikkvariabel",age,gammel,Andel over 65 år,Fylkesnr,Fylke
0,693494,5460.0,87347,12.595206,3,Oslo
1,479892,5460.0,71967,14.996499,11,Rogaland
2,265238,5460.0,52367,19.743400,15,Møre og Romsdal
3,241235,5460.0,50191,20.805853,18,Nordland
4,0,5460.0,0,NaN,21,Svalbard


In [97]:
df

,"07459: Befolkning, etter region, alder, år og statistikkvariabel",age,gammel,Andel over 65 år,Fylkesnr,Fylke
0,693494,5460.0,87347,12.595206,3,Oslo
1,479892,5460.0,71967,14.996499,11,Rogaland
2,265238,5460.0,52367,19.743400,15,Møre og Romsdal
3,241235,5460.0,50191,20.805853,18,Nordland
4,0,5460.0,0,NaN,21,Svalbard
5,1241165,5460.0,217911,17.556973,30,Viken
6,371385,5460.0,82379,22.181564,34,Innlandet
7,419396,5460.0,84921,20.248405,38,Vestfold og Telemark
8,307231,5460.0,54338,17.686366,42,Agder
9,636531,5460.0,111289,17.483673,46,Vestland


In [119]:
legend = [i for i in df["Fylke"].dropna().unique()]
legend.sort()

fig = px.bar(df.dropna(), 
             x='Andel over 65 år', y='Fylke', 
             orientation='h',
             category_orders={"Fylke": legend},
             height=500,
             title = "<b>Andel av befolkningen over 65 år</b><br>Datakilde: ssb.no.",
             color='07459: Befolkning, etter region, alder, år og statistikkvariabel',
             hover_name="Fylke",
             hover_data=["Andel over 65 år"],
             template = "plotly_white", 
             #labels = {"Vekstrate": "daglig vekst (%)", "- 1 uke": "Daglig vekst for en uke siden", "text": "Dager før dobling med dagens vekst"},
            )


fig.layout.coloraxis.showscale = False

fig.update_layout(
    yaxis=dict(
        separatethousands=True,
        categoryorder = 'total ascending',
        title_text="",
        fixedrange=True,
    ), xaxis=dict(fixedrange=True,
                 title_text=""),
    title_font=dict(family="Helvetica", size=18),
        margin=dict(b=150)) #bottom margin

fig.update_layout(autosize=True,
    margin=dict(l=100, r=100, t=150, b=0),
    paper_bgcolor='rgba(0, 0, 0, 0)'
    #plot_bgcolor='rgb(248, 248, 255)'    
                 )

#plotly.offline.plot(fig, filename='COVID_moment_fylke.html')
#py.plot(fig, filename='COVID19_vekstmomentum_fylker.html', auto_open=False)
fig.show()